In [3]:
from dataset import GTSequenceDataset
from torch.utils.data import DataLoader, ConcatDataset

SEQ_IN_LEN = 30
SEQ_OUT_LEN = 20
SEQ_TOTAL_LEN = 50
BATCH_SIZE = 1024
STEPS = 4
NOISE_COEFFICIENT = 0
NOISE_PROB = 0
NOISE_COEFFICIENT = 0.15
NOISE_PROB = 0.2

BASE_DIR = '../../Datasets/'
# train_dataset = ConcatDataset([
#     GTSequenceDataset.from_roots([f'{BASE_DIR}DanceTrack/train'], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB),
#     GTSequenceDataset.from_roots([f'{BASE_DIR}MOT20/train'], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB),
#     GTSequenceDataset.from_roots([f'{BASE_DIR}MOT17/train'], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB),
# ])
# val_dataset = ConcatDataset([
#     GTSequenceDataset.from_roots([f'{BASE_DIR}DanceTrack/val'], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB),
#     GTSequenceDataset.from_roots([f'{BASE_DIR}MOT20/val'], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB),
#     GTSequenceDataset.from_roots([f'{BASE_DIR}MOT17/val'], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB),
# ])
train_dataset = GTSequenceDataset.from_roots([
    # f'{BASE_DIR}/SportsMOT/train',
    # f'{BASE_DIR}DanceTrack/train',
    f'{BASE_DIR}MOT17/train',
    # f'{BASE_DIR}MOT20/train'
], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB)

val_dataset = GTSequenceDataset.from_roots([
    # f'{BASE_DIR}/SportsMOT/val',
    # f'{BASE_DIR}DanceTrack/val',
    f'{BASE_DIR}MOT17/val',
    # f'{BASE_DIR}MOT20/val'
# ], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=0, noise_prob=0)
], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

print(f'Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}')

Train samples: 15072, Val samples: 28163


In [ ]:
from dataset import GTSequenceDataset
from torch.utils.data import DataLoader, ConcatDataset
from transformer_encoder import MotionTransformer
from loss import LossFunction
from torch import optim
import math, torch, torch.nn as nn
import torch.nn.functional as F
import random
import os



SEQ_IN_LEN = 30
SEQ_OUT_LEN = 20
SEQ_TOTAL_LEN = 50
BATCH_SIZE = 1024
STEPS = 4
NOISE_COEFFICIENT = 0
NOISE_PROB = 0
NOISE_COEFFICIENT = 0.15
LR = 5e-4
DEVICE = 'cuda'
PARAMS = [[35, 0], [15, 0.2], [10, 0.4], [10, 0.6]]

model = MotionTransformer(
    input_dim=13,
    output_dim=5,
    # d_model=128,
    d_model=512,
    # nhead=16,
    nhead=32,
    num_layers=1,
    dim_ff=512,
    dropout=0.15,
).to(DEVICE)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
optimizer = optim.AdamW(
    model.parameters(),
    lr=LR,
    betas=(0.9, 0.999),
    weight_decay=1e-4,
    eps=1e-8
)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=sum([p[0] for p in PARAMS]) + 1)

log_file = open('file.log', 'w')
log_file.close()
# NOISE_PROB = 0.2

for param in PARAMS:
    NUM_EPOCHS, NOISE_PROB = param
    if NOISE_PROB > 0:
        criterion = LossFunction(loss1_coeff=0.5, loss2_coeff=0, loss3_coeff=1, loss4_coeff=0)
    else:
        criterion = LossFunction(loss1_coeff=1, loss2_coeff=0, loss3_coeff=0, loss4_coeff=0)
    best_val_loss = float("inf")

    BASE_DIR = '../../Datasets/'
    train_dataset = GTSequenceDataset.from_roots([
        f'{BASE_DIR}/SportsMOT/train',
        f'{BASE_DIR}DanceTrack/train',
        f'{BASE_DIR}MOT17/train',
        f'{BASE_DIR}MOT20/train'
    ], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB)
    val_dataset = GTSequenceDataset.from_roots([
        f'{BASE_DIR}/SportsMOT/val',
        f'{BASE_DIR}DanceTrack/val',
        f'{BASE_DIR}MOT17/val',
        f'{BASE_DIR}MOT20/val'
    # ], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=0, noise_prob=0)
    ], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

    for epoch in range(1, NUM_EPOCHS + 1):
        train_loss = model.train_one_epoch(train_loader, optimizer, criterion)
        val_loss = model.evaluate(val_loader, criterion)

        scheduler.step()

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            model.save_weight('pretrained/transformer-encoder-d256-ff512-nh32-1l-n3.pth')
            # model.save_weight('pretrained/transformer-encoder-d512-ff512-2l.pth')

        current_lr = scheduler.get_last_lr()[0]
        print(f"Epoch {epoch}: Train Loss = {train_loss:.8f}, Val Loss = {val_loss:.8f}, LR = {current_lr:.8f}")
        log_file = open('file.log', 'a')
        log_file.write(f"Epoch {epoch}: Train Loss = {train_loss:.8f}, Val Loss = {val_loss:.8f}, LR = {current_lr:.8f}\n")
        log_file.close()

    print("Training complete. Best Val Loss:", best_val_loss)

    

    

Model parameters: 1,911,813
Epoch 1: Train Loss = 0.60551686, Val Loss = 0.17987538, LR = 0.00049976
Epoch 2: Train Loss = 0.19389952, Val Loss = 0.10858391, LR = 0.00049902
Epoch 3: Train Loss = 0.13091740, Val Loss = 0.09556698, LR = 0.00049780
Epoch 4: Train Loss = 0.10724405, Val Loss = 0.08109847, LR = 0.00049609
Epoch 5: Train Loss = 0.08804472, Val Loss = 0.07756049, LR = 0.00049391
Epoch 6: Train Loss = 0.07673899, Val Loss = 0.07491766, LR = 0.00049124
Epoch 7: Train Loss = 0.06828707, Val Loss = 0.06553560, LR = 0.00048810
Epoch 8: Train Loss = 0.06297994, Val Loss = 0.06697503, LR = 0.00048450
Epoch 9: Train Loss = 0.05983980, Val Loss = 0.06417410, LR = 0.00048044
Epoch 10: Train Loss = 0.06022786, Val Loss = 0.07111049, LR = 0.00047592
Epoch 11: Train Loss = 0.06114988, Val Loss = 0.06295310, LR = 0.00047097
Epoch 12: Train Loss = 0.05478626, Val Loss = 0.06045165, LR = 0.00046558
Epoch 13: Train Loss = 0.05296187, Val Loss = 0.05958388, LR = 0.00045977
Epoch 14: Train Los

In [1]:
from transformer_encoder import MotionTransformer
from loss import LossFunction
from torch import optim
import math, torch, torch.nn as nn
import torch.nn.functional as F
import random
import os


DEVICE = 'cuda'
model = MotionTransformer(
    input_dim=13,
    output_dim=5,
    d_model=256,
    # d_model=512,
    # nhead=16,
    nhead=32,
    num_layers=6,
    dim_ff=512,
    dropout=0.15,
).to(DEVICE)
criterion = LossFunction(loss1_coeff=1, loss2_coeff=0, loss3_coeff=0, loss4_coeff=0)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")


Model parameters: 3,247,621


In [17]:
model.load_weight('pretrained/transformer-encoder-d256-ff512-nh32-1l-n3.pth')
# model.load_weight('pretrained/transformer-encoder-d256-ff512-nh32-1l-n.pth')
# model.load_weight('pretrained/transformer-encoder-d256-ff512-2l.pth')
# model.load_weight('pretrained/transformer-encoder-d256-ff512-6l-ft.pth')
# model.load_weight('pretrained/transformer-encoder-d512-ff512-2l-ft.pth')
# model.load_weight('pretrained/transformer-encoder-d512-ff512-6l.pth')

In [19]:
model.evaluate(val_loader, criterion)

0.16076720340384376

In [39]:
criterion1 = LossFunction(loss1_coeff=1, loss2_coeff=0, loss3_coeff=0, loss4_coeff=0)
# criterion2 = LossFunction(loss1_coeff=0, loss2_coeff=0, loss3_coeff=0, loss4_coeff=0)


In [40]:
model.evaluate(val_loader, criterion1)

0.057145387296461396

In [ ]:
LR = 1e-5
NUM_EPOCHS = 20

optimizer = optim.AdamW(
    model.parameters(),
    lr=LR,
    betas=(0.9, 0.999),
    weight_decay=1e-4,
    eps=1e-8
)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(
#         optimizer,
#         mode='min',
#         factor=0.5,
#         patience= 11,  # Reduce LR before early stopping
#         verbose=True,
#         min_lr=LR * 0.001
#     )
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS + 1)
best_val_loss = float("inf")
log_file = open('file.log', 'w')
log_file.close()

for epoch in range(1, NUM_EPOCHS + 1):
    train_loss = model.train_one_epoch(train_loader, optimizer, criterion)
    val_loss = model.evaluate(val_loader, criterion)

    scheduler.step()

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        model.save_weight('pretrained/transformer-encoder-d256-ff512-nh32-1l-n3.pth')
        # model.save_weight('pretrained/transformer-encoder-d512-ff512-2l.pth')

    current_lr = scheduler.get_last_lr()[0]
    print(f"Epoch {epoch}: Train Loss = {train_loss:.8f}, Val Loss = {val_loss:.8f}, LR = {current_lr:.8f}")
    log_file = open('file.log', 'a')
    log_file.write(f"Epoch {epoch}: Train Loss = {train_loss:.8f}, Val Loss = {val_loss:.8f}, LR = {current_lr:.8f}\n")
    log_file.close()

print("Training complete. Best Val Loss:", best_val_loss)

Epoch 1: Train Loss = 0.17073468, Val Loss = 0.15598132, LR = 0.00000994
Epoch 2: Train Loss = 0.16916021, Val Loss = 0.15502857, LR = 0.00000978
Epoch 3: Train Loss = 0.16823071, Val Loss = 0.15454584, LR = 0.00000950
Epoch 4: Train Loss = 0.16758093, Val Loss = 0.15407908, LR = 0.00000913
Epoch 5: Train Loss = 0.16697631, Val Loss = 0.15332996, LR = 0.00000867
Epoch 6: Train Loss = 0.16642478, Val Loss = 0.15303068, LR = 0.00000812
Epoch 7: Train Loss = 0.16596075, Val Loss = 0.15237219, LR = 0.00000750


In [15]:
log_file = open('file.log', 'w')
log_file.close()

In [31]:
model.save_weight('pretrained/transformer-encoder-d128-ff512-nh32-1l.pth')
# model.save_weight('pretrained/transformer-encoder-d256-ff512-2l-n2.pth')
# model.load_weight('pretrained/transformer-encoder-d512-ff512-2l-.pth')
# model.load_weight('pretrained/transformer-encoder-d256-ff512-2l-test2.pth')

In [44]:
# a, b, c, d = next(iter(train_loader))
a, b, c, d = next(iter(val_loader))
a = a.cuda()
b = b.cuda()
c = c.cuda()
d = d.cuda()
o = model.forward(a, b[:, :-1])[13]

In [45]:
o[:, 4]

tensor([0.8831, 0.8832, 0.8835, 0.8836, 0.8837, 0.8836, 0.8832, 0.8830, 0.8832,
        0.8832, 0.8834, 0.8835, 0.8830, 0.8827, 0.8824, 0.8824, 0.8829, 0.8831,
        0.8833], device='cuda:0', grad_fn=<SelectBackward0>)

In [7]:
g = torch.tensor([[[0.1976, 0.0841, 0.0298, 0.0943]]]).cuda()
h = torch.tensor([[[0.1991, 0.0818, 0.0281, 0.0977]]]).cuda()

criterion.iou(g, h)

tensor([[[0.8621]]], device='cuda:0')

In [8]:
torch.diagonal(criterion.iou(d[:, :, :4], o[:, :, :4]), dim1=1, dim2=2).mean()

IndexError: too many indices for tensor of dimension 2

In [46]:
o[:, :4]

tensor([[0.8457, 0.0280, 0.0290, 0.0544],
        [0.8457, 0.0280, 0.0290, 0.0544],
        [0.8457, 0.0285, 0.0290, 0.0555],
        [0.8457, 0.0285, 0.0290, 0.0555],
        [0.8457, 0.0285, 0.0290, 0.0555],
        [0.8457, 0.0291, 0.0290, 0.0566],
        [0.8457, 0.0291, 0.0290, 0.0566],
        [0.8457, 0.0291, 0.0290, 0.0567],
        [0.8457, 0.0297, 0.0290, 0.0578],
        [0.8453, 0.0297, 0.0281, 0.0578],
        [0.8457, 0.0297, 0.0273, 0.0578],
        [0.8457, 0.0297, 0.0273, 0.0578],
        [0.8461, 0.0297, 0.0264, 0.0578],
        [0.8466, 0.0302, 0.0256, 0.0589],
        [0.8466, 0.0302, 0.0256, 0.0589],
        [0.8470, 0.0302, 0.0247, 0.0589],
        [0.8470, 0.0303, 0.0247, 0.0589],
        [0.8474, 0.0303, 0.0239, 0.0589],
        [0.8478, 0.0308, 0.0230, 0.0601]], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [47]:
d[13, 1:, :4]

tensor([[0.8457, 0.0284, 0.0290, 0.0545],
        [0.8457, 0.0290, 0.0290, 0.0557],
        [0.8457, 0.0290, 0.0290, 0.0557],
        [0.8457, 0.0290, 0.0290, 0.0557],
        [0.8457, 0.0295, 0.0290, 0.0568],
        [0.8457, 0.0295, 0.0290, 0.0568],
        [0.8457, 0.0295, 0.0290, 0.0568],
        [0.8457, 0.0301, 0.0290, 0.0580],
        [0.8453, 0.0301, 0.0281, 0.0580],
        [0.8457, 0.0301, 0.0273, 0.0580],
        [0.8457, 0.0301, 0.0273, 0.0580],
        [0.8461, 0.0301, 0.0264, 0.0580],
        [0.8465, 0.0307, 0.0256, 0.0591],
        [0.8465, 0.0307, 0.0256, 0.0591],
        [0.8470, 0.0307, 0.0247, 0.0591],
        [0.8470, 0.0307, 0.0247, 0.0591],
        [0.8474, 0.0307, 0.0239, 0.0591],
        [0.8478, 0.0312, 0.0230, 0.0602],
        [0.8478, 0.0307, 0.0230, 0.0591]], device='cuda:0')

In [25]:
c[13, :, :4]

tensor([[0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0.1389],
        [0.1747, 0.0491, 0.0234, 0

In [18]:
import torch
from dataset import GTSequenceDataset

# SEQ_PATH = '../../Datasets/MOT20/val/MOT20-01/'
SEQ_PATH = '../../Datasets/MOT17/val/MOT17-05-FRCNN//'
# SEQ_PATH = '../../Datasets/DanceTrack/val/dancetrack0018/'
# SEQ_PATH = '../../Datasets/DanceTrack/train/dancetrack0006//'
# SEQ_PATH = '../../Datasets/SportsMOT/val/v_0kUtTtmLaJA_c004//'
SEQ_IN_LEN = 50
SEQ_OUT_LEN = 2
SEQ_TOTAL_LEN = SEQ_IN_LEN + SEQ_OUT_LEN
BATCH_SIZE = 512
STEPS = 4

# d = GTSequenceDataset.from_sequence(SEQ_PATH, seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, noise_prob=0, noise_coeff=0)
d = GTSequenceDataset.from_sequence(SEQ_PATH, seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=0.15, noise_prob=0.4)
sources = torch.tensor(d.sources).to(DEVICE)[:BATCH_SIZE]
# targets = torch.tensor(d.targets).to(DEVICE)[:BATCH_SIZE]
gt_sources = torch.tensor(d.gt_sources).to(DEVICE)[:BATCH_SIZE]
targets = torch.tensor(d.gt_targets).to(DEVICE)[:BATCH_SIZE]
# gt_targets = torch.tensor(d.gt_targets).to(DEVICE)[:BATCH_SIZE]

# o = model.inference(sources, targets, num_steps=targets.size(1) - 1)
o = model.forward(sources, targets[:, :-1])

o[:, :, 0] *= d.image_width.item()
o[:, :, 2] *= d.image_width.item()
o[:, :, 1] *= d.image_height.item()
o[:, :, 3] *= d.image_height.item()
targets[:, :, 0] *= d.image_width.item()
targets[:, :, 2] *= d.image_width.item()
targets[:, :, 1] *= d.image_height.item()
targets[:, :, 3] *= d.image_height.item()
t_ = targets[:, 1:, :4]
sources[:, :, 0] *= d.image_width.item()
sources[:, :, 2] *= d.image_width.item()
sources[:, :, 1] *= d.image_height.item()
sources[:, :, 3] *= d.image_height.item()
o_ = o[:, :, :4]

index = 9
# t[index], o[index]
# f = (t[index] - o[index]).abs()[:, 3].mean()
# torch.diagonal(criterion.iou(t, o), dim1=1, dim2=2).mean()
# criterion.iou(t_, o_).mean()
criterion.ciou(t_, o_).mean()
# f = (t_ - o_).abs().mean()
# f


tensor(0.9187, device='cuda:0', grad_fn=<MeanBackward0>)

In [15]:
t_[0]

tensor([[643.0000, 416.0000,  40.0000, 122.0000]], device='cuda:0')

In [16]:
o[0]

tensor([[639.8837, 416.5098,  40.3046, 121.7041,   0.9208]], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [ ]:
o[:, :, 4]

tensor(0.8833, device='cuda:0', grad_fn=<MeanBackward0>)

In [15]:
a = torch.tensor(
    [
        [
            [10,10,10,10],
            [30,30,40,40],
        ],
        [
            [30,30,20,20],
            [30,30,20,20]
        ],
        [
            [50,50,10,10],
            [40,40,20,20]
        ]
    ],
)
b = torch.tensor(
    [
        [
            [10,10,10,10],
            [30,30,40,40],
        ],
        [
            [30,30,20,20],
            [30,30,20,20]
        ],
        [
            [50,50,10,10],
            [40,40,20,20]
        ]
    ],
)

criterion.ciou(a, b)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [9]:
o[:, :, 4]

tensor([[0.9185],
        [0.9173],
        [0.9353],
        [0.9617],
        [0.9837],
        [0.9441],
        [0.9207],
        [0.8818],
        [0.8690],
        [0.8990],
        [0.8124],
        [0.9717],
        [0.9776],
        [0.9802],
        [0.9065],
        [0.8873],
        [0.9421],
        [0.9235],
        [0.9258],
        [0.9445],
        [0.9869],
        [0.8960],
        [0.9319],
        [0.9422],
        [0.9389],
        [0.9459],
        [0.9354],
        [0.9902],
        [0.9693],
        [0.8957],
        [0.9465],
        [0.9469],
        [0.9175],
        [0.9542],
        [0.9261],
        [0.8819],
        [0.9333],
        [0.9219],
        [0.8752],
        [0.9421],
        [0.9532],
        [0.9610],
        [0.9848],
        [0.9669],
        [0.9172],
        [0.9139],
        [0.9219],
        [0.9324],
        [0.9209],
        [0.9316],
        [0.9463],
        [0.9569],
        [0.8696],
        [0.9285],
        [0.9420],
        [0

In [10]:
targets[:, :, 4]

tensor([[ 0.0039,  0.0035],
        [ 0.0043,  0.0031],
        [ 0.0070,  0.0016],
        ...,
        [-0.0070, -0.0188],
        [ 0.0148, -0.0070],
        [-0.0023, -0.0047]], device='cuda:0')

In [38]:
gt_sources[0][0]

tensor([0.4023, 0.4671, 0.0672, 0.2343, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 1.0000], device='cuda:0')